<a href="https://colab.research.google.com/github/CDSGROUP11/Project-10/blob/UI_Feature/MainUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -qq install langchain_openai
!pip -qq install PyPDF2
!pip -qq install langchain_community
!pip -qq install faiss-gpu-cu11
!pip -qq install streamlit pyngrok
!pip -qq install streamlit_modal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 549.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 43.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.

In [ ]:
from langchain_openai import ChatOpenAI
from PyPDF2 import PdfReader
import os

In [ ]:
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import streamlit as st

In [ ]:
from pyngrok import ngrok
!ngrok config add-authtoken <your token>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile main.py
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
from datetime import datetime
import os
from streamlit_modal import Modal

# Ensure GITHUB_TOKEN is set as an environment variable for use in ChatOpenAI
github_token = <your token>
os.environ['GITHUB_TOKEN'] = github_token
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4.1-nano"

def get_pdf_text(pdf):
    text = ""
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
        text+= page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 10000,
    chunk_overlap = 100
    )
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks):
    embeddings = OpenAIEmbeddings(
        model='text-embedding-3-small',
        dimensions=1536,
        base_url = endpoint,
        api_key= github_token)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")


def get_conversation_chain():
    model = ChatOpenAI(
    base_url = endpoint,
    api_key= github_token,
    model=model_name,
    temperature=0.1
    )
    chat_prompt_template = ChatPromptTemplate([
         ('system', 'You are good resume parser expert and helps in answering questions. Answer the question from the provided resume. Make sure to provide all the details, if the answer is not in the provided resume just reply back, "Answer is not available in provided resume", dont provide the wrong answers or dont hallucinate the answers.\n Resume: \n {resume}?'),
         ('human', 'Question: \n {question}\nAnswer:')
        ])
    chain = chat_prompt_template|model|StrOutputParser()
    return chain

def notice_period_prompt():
    model = ChatOpenAI(
    base_url = endpoint,
    api_key= github_token,
    model=model_name,
    temperature=0.1
    )
    chat_prompt_template = ChatPromptTemplate([
         ('system', 'You are good and helpful assistant and helps in answering questions. Find out the notice period from public available sources for most recent organization provided in resume. Make sure to provide to provide accurate details and provide the source link at the end of answer, dont provide the wrong answers or dont hallucinate the answers. \n Resume: \n {resume}?'),
         ('human', 'Question: \n Notice Period of the most recent job organization the candidate working for. \nAnswer:')
        ])
    chain = chat_prompt_template|model|StrOutputParser()
    return chain

def handle_notice_period():
    user_question = "Notice Period of the most recent job organization the candidate working for"
    embeddings = OpenAIEmbeddings(
        model='text-embedding-3-small',
        dimensions=1536,
        base_url = endpoint,
        api_key= github_token,)
    new_db = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)
    chain = notice_period_prompt()
    response = chain.invoke({
        "resume": docs
    })
    return response

def handle_user_input(user_question):
    embeddings = OpenAIEmbeddings(
        model='text-embedding-3-small',
        dimensions=1536,
        base_url = endpoint,
        api_key= github_token,)
    new_db = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)
    chain = get_conversation_chain()
    response = chain.invoke({
        "resume": docs,
        "question": user_question
    })
    return response

st.set_page_config("Pre-Screening Intelligence: TalAI", layout="wide")

left_col, middle_col, right_col = st.columns([1, 1, 1])

# Initialize modal
resume_modal = Modal("📂 Resume Tools", key="resume-tools-modal", max_width="700px")

# with left_col:
#     st.header("📂 Resume Tools")
#     pdf_doc = st.file_uploader("Upload PDF Resume")
#     if st.button("Submit and Process") and pdf_doc:
#         with st.spinner("Processing..."):
#             raw_text = get_pdf_text(pdf_doc)
#             text_chunks = get_text_chunks(raw_text)
#             get_vector_store(text_chunks)
#             st.success("Resume processed and indexed!")
#             st.session_state.chat_history = []

section_list = [
    'Work Experience',
    'Education',
    'Skills',
    'Academic Projects',
    'Certification, Award & Recognition',
    'Career Objectives or Summary',
    'Personal Details & Hobbies'
]

with left_col:
    # Upload button triggers modal
    if st.button("📤 Upload Resumes"):
        resume_modal.open()

    # Modal content
    if resume_modal.is_open():
        with resume_modal.container():
            st.markdown("""
            <div style="display: flex; flex-direction: column; align-items: center; text-align: center;">
                <img src="https://raw.githubusercontent.com/surabhi13gupta/LangChains/main/TalAI.png" width="60" style="border-radius: 50%; margin-bottom: 1rem;">
                <h4 style="margin-bottom: 0;">TalAI ResumeTools</h4>
                <p style="margin-top: 0;">Upload your resume to extract insights, skills, and suggestions.</p>
                </div>""", unsafe_allow_html=True)

            # Resume upload
            uploaded_file = st.file_uploader("Upload Resume (PDF/DOCX are allowed)", type=["pdf", "docx"])

            if st.button("Submit and Process") and uploaded_file:
                with st.spinner("Processing..."):
                    raw_text = get_pdf_text(uploaded_file)
                    text_chunks = get_text_chunks(raw_text)
                    get_vector_store(text_chunks)
                    st.success("Resume uploaded, processed and indexed!")
                    st.session_state.chat_history = []
                    st.button("✅ Close", on_click=lambda: resume_modal.close())
            else:
                st.info("Awaiting file upload...")

    st.header("📊 Top 5 Resumes")

    # Job description input
    job_description = st.text_area(
        "📝 Job Description",
        height=200,
        placeholder="Paste the job role or requirements here..."
    )

    for section in section_list:
        with st.expander(f"📂 {section}"):
            st.text_area(f"✏️ Edit or review: {section}", height=150, placeholder=f"Enter details for {section}...")

    # Submit button
    if st.button("🚀 Match Resumes") and job_description:
        with st.spinner("Matching resumes..."):
            # Placeholder for vector search logic
            top_matches = [
                "**1. Jane Doe** — ML Engineer, 5 yrs exp, NLP-heavy projects",
                "**2. Ravi Kumar** — Time Series Specialist, fintech background",
                "**3. Aisha Rahman** — GenAI pipeline builder, LangChain expert",
                "**4. Leo Zhang** — Resume parsing wizard, UX-focused",
                "**5. Sara Ali** — Dashboard designer, Streamlit + LLM integration"
            ]

            # Display results
            st.markdown("### 🏆 Top Resume Matches")
            for match in top_matches:
                st.markdown(f"- {match}")

with middle_col:
    st.header("🎥 Video Summarization")
    st.button("Summarize Video")

with right_col:
    st.header("💬 Chat with TalAI")
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    for msg in st.session_state.chat_history:
        role = "user" if msg["role"] == "user" else "assistant"
        st.chat_message(role).write(msg["content"])

    chat_container = st.container()

    # Handle input first
    first_question = "Candidate Name, total years of experience and recent job organisation"
    st.session_state.chat_history.append({
            "role": "user",
            "content": first_question
        })
    recent_job = handle_user_input(first_question)
    st.session_state.chat_history.append({
        "role": "assistant",
        "content": recent_job
    })

    st.session_state.chat_history.append({
            "role": "user",
            "content": "Notice Period information for recent company"
        })
    notice_response = handle_notice_period()
    st.session_state.chat_history.append({
        "role": "assistant",
        "content": notice_response
    })

    user_question = st.chat_input("Ask TalAI about the selected resume...")
    if user_question:
        st.session_state.chat_history.append({
            "role": "user",
            "content": user_question
        })
        response = handle_user_input(user_question)
        st.session_state.chat_history.append({
            "role": "assistant",
            "content": response
        })

    # Render all messages inside chat_container
    with chat_container:
        for msg in (st.session_state.chat_history):  # Latest at top
            role = "user" if msg["role"] == "user" else "assistant"
            prefix = "You: " if role == "user" else "TalAI: "
            st.chat_message(role).write(prefix + msg["content"])

Writing main.py


In [ ]:
public_url = ngrok.connect(addr=8501)

print(f"Streamlit URL: {public_url}")

ERROR:pyngrok.process.ngrok:t=2025-08-21T17:00:48+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-21T17:00:48+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [ ]:
!streamlit run main.py&




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.147.78.251:8501

  Stopping...


load_qa_chain is a LangChain helper that loads a Chain designed for answering questions based on provided documents. It wraps a language model (LLM) with a prompt template and logic for combining document context and user queries